In [69]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
# Random
import random
random.seed = 42
# Thread management (sleep purposes)
import time
# Operating System management
import os
import selenium
# Selenium specific libraries
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

import pickle

In [70]:
soup = BeautifulSoup(requests.get('https://josemamerto.blogspot.com/search?updated-max=2024-05-06T00:51:00-07:00&max-results=11').content, 'html.parser')

titles = []
links = []

In [71]:
titulos = soup.find_all('h3', class_='post-title entry-title')
for i in titulos:
    print(i.text)
    print(i.a['href'])
    print('----------------------')
    titles.append(i.text)
    links.append(i.a['href'])


Espacios Vectoriales

https://josemamerto.blogspot.com/2024/05/espacios-vectoriales.html
----------------------

Pero esto qué es

https://josemamerto.blogspot.com/2024/05/pero-esto-que-es.html
----------------------

El pelucas

https://josemamerto.blogspot.com/2024/03/blog-post.html
----------------------


In [72]:
soup = BeautifulSoup(requests.get('https://josemamerto.blogspot.com/').content, 'html.parser')

text = soup.find('h3', class_='post-title').text
enlace = soup.find('h3', class_='post-title').a['href']
print(text)
print(f'\n{enlace}')
print('----------------------')
titles.append(text)
links.append(enlace)    

texts = soup.find_all('h3', class_='post-title entry-title')

for i in texts:
    print(i.text)
    print(i.a['href'])
    print('----------------------')
    titles.append(i.text)
    links.append(i.a['href'])

NLP (y IV) Creando un Buscador de Pisos con Similitud de Coseno y modelo LLaMA

https://josemamerto.blogspot.com/2024/05/nlp-y-iv-creando-un-buscador-de-pisos.html
----------------------

NLP (III)  Explorando Temas Ocultos: Topic Modeling y Word Embeddings

https://josemamerto.blogspot.com/2024/05/nlp-iii-explorando-temas-ocultos-topic.html
----------------------

NLP (II) Domando los Datos: Preprocesamiento y Almacenamiento en DataFrame

https://josemamerto.blogspot.com/2024/05/nlp-ii-domando-los-datos.html
----------------------

NLP (I) Web Scraping

https://josemamerto.blogspot.com/2024/05/nlp-i-web-scraping.html
----------------------


In [73]:
#limpiar titles, eliminar saltos de linea
titles = [re.sub('\n', '', i) for i in titles]

In [74]:
textos = {titulo: [] for titulo in titles}
for titulo, link in zip(titles, links):
    soup = BeautifulSoup(requests.get(f'{link}').content, 'html.parser')
    articulo_div = soup.find('div', class_ = 'post-body entry-content float-container')
    articulos = articulo_div.find_all('p')  # find all 'p' tags in the div
    for articulo in articulos:
        textos[titulo].append(articulo.text)

In [75]:
for titulo in textos:
    textos[titulo] = ' '.join(textos[titulo])

In [76]:
df = pd.DataFrame(list(textos.items()), columns=['Title', 'Text'])
df

,Title,Text
0,Espacios Vectoriales,Explorando los Espacios Vectoriales: Fundamen...
1,Pero esto qué es,Acá en este bloggaso vamos a presentar unos d...
2,El pelucas,
3,NLP (y IV) Creando un Buscador de Pisos con Si...,"En esta última parte de nuestro análisis, cons..."
4,NLP (III) Explorando Temas Ocultos: Topic Mod...,"En el análisis de texto, el topic modeling es ..."
5,NLP (II) Domando los Datos: Preprocesamiento y...,"En el fascinante mundo del análisis de datos,..."
6,NLP (I) Web Scraping,¿Alguna vez te has preguntado cómo se recopil...


In [77]:
import spacy
import unicodedata

nlp=spacy.load('es_core_news_sm')

def normalize_document(doc):

    doc = unicodedata.normalize('NFKD', doc).encode('ASCII', 'ignore').decode('utf-8')

    # tokenizamos el texto
    tokens = nlp(doc)
    # quitamos puntuación/espacios/stop words y cogemos el lema
    lemmas = [t.lemma_ for t in tokens if not t.is_punct and not t.is_space and not t.is_stop]
    doc = ' '.join(lemmas)
    return doc

def normalize_corpus(corpus):
    """Normaliza un corpus de documentos aplicando al función de normalización
    normalize_document() a cada documento de la lista pasada como argumento"""   
    return [normalize_document(text) for text in corpus]

corpus = df['Text'].tolist()
corpus = normalize_corpus(corpus)

In [78]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# usamos características tf-idf para LSA.
tfidf_vectorizer = TfidfVectorizer(min_df=2)
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [79]:
def print_top_words(model, feature_names, n_top_words):
    """Función auxiliar para mostrar los términos más importantes
    de cada topic"""
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [80]:
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

# Ajustamos el modelo LSA
lsa = TruncatedSVD(n_components=3).fit(tfidf)

print("\nTopics en modelo LSA:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lsa, tfidf_feature_names, 5)


Topics en modelo LSA:
Topic #0: dato piso paso web scraping
Topic #1: word texto paso embeddings analisis
Topic #2: vector propiedad aplicación conjunto word



In [82]:
# assuming lsa and tfidf are defined and have the correct dimensions
topic_values = np.round(lsa.transform(tfidf), 4)

# convert the array to a DataFrame
topic_df = pd.DataFrame(topic_values, columns=['Topic 0', 'Topic 1', 'Topic 2'])

# concatenate the DataFrames
df = pd.concat([df, topic_df], axis=1)

df

,Title,Text,Topic 0,Topic 1,Topic 2
0,Espacios Vectoriales,Explorando los Espacios Vectoriales: Fundamen...,0.3631,-0.5178,0.6996
1,Pero esto qué es,Acá en este bloggaso vamos a presentar unos d...,0.6585,-0.4431,-0.1216
2,El pelucas,,0.0000,0.0000,0.0000
3,NLP (y IV) Creando un Buscador de Pisos con Si...,"En esta última parte de nuestro análisis, cons...",0.6331,0.0632,0.0645
4,NLP (III) Explorando Temas Ocultos: Topic Mod...,"En el análisis de texto, el topic modeling es ...",0.5704,0.6455,0.3228
5,NLP (II) Domando los Datos: Preprocesamiento y...,"En el fascinante mundo del análisis de datos,...",0.7538,0.2713,-0.1401
6,NLP (I) Web Scraping,¿Alguna vez te has preguntado cómo se recopil...,0.5397,-0.2464,-0.5435


In [84]:
keys = links
comentarios = {key: {} for key in keys}

for i in keys:
    soup = BeautifulSoup(requests.get(f'{i}').content, 'html.parser')
    comentario = soup.find_all('li', class_ = 'comment')
    for j in comentario:
        try :
            comments = j.find('p', class_ = 'comment-content')
            user = j.find('cite', class_ = 'user')
            print(i)
            print(user.text)
            print(comments.text)
            print('----------------------')
            comentarios[i][user.text] = comments.text
        except AttributeError:
            comments = 'No hay comentarios'
            print(i)
            print(comments)

https://josemamerto.blogspot.com/2024/05/espacios-vectoriales.html
alex es mi lider
ché me cague a trompadas de risa con el content, publicá más xfiiiiiiiii
----------------------
https://josemamerto.blogspot.com/2024/05/espacios-vectoriales.html
Elseroculto
空间向量是线性代数研究的基石，这一学科在物理学、工程学、计算机科学和经济学等领域有广泛的应用。在这篇博客中，我们将深入探讨空间向量的世界，探索其基础、特性和应用。什么是空间向量？空间向量是一种数学结构，由一组称为向量的元素以及两种操作组成，即向量加法和标量乘法，满足某些基本性质。正式地，空间向量𝑉V 在域𝐹F 上是一个非空元素集合，称为向量，以及两种在𝑉V 上定义的操作：向量加法和标量乘法，这些操作满足八条公理，规定了它们必须满足的基本性质。特性和示例空间向量可以具有多种特性，其中一些包括对加法和标量乘法封闭、存在加法单位元素、存在加法逆元素和操作的结合律。常见的空间向量示例包括欧几里得空间𝑅𝑛，其中包含所有n-维向量，其坐标为实数，以及𝑃𝑛多项式空间，其多项式的次数不超过n。在现实生活中的应用空间向量在现实世界中有广泛的应用。在物理学中，它们用于建模力、力矩和电磁场。在工程学中，它们应用于结构分析、电路和控制系统。在计算机科学中，它们用于图像处理、计算机图形学和机器学习。甚至在经济学中，它们用于建模经济变量之间的关系和优化财务决策。高级特性和定理除了基本性质外，空间向量还表现出一系列更高级的特性和重要定理。例如，共同基础定理规定所有空间向量都有一个基础，即一组线性独立的向量，它们生成整个空间。秩定理告诉我们矩阵的秩等于可以从其列中获得的线性独立向量的最大数量。结论空间向量是数学和应用科学中强大而多才多艺的工具。它们的研究不仅提供了对空间结构和线性变换的深入理解，而且在各种领域都有实际应用。无论是对亚原子粒子运动建模还是设计人工智能算法，空间向量仍然是我们对周围世界理解的不可或缺的一部分。希望这次探索给您带来了对空间向量重要性和美妙之处的清晰认识！如果您有任

In [21]:
comentarios

{'https://josemamerto.blogspot.com/2024/05/espacios-vectoriales.html': {'alex es mi lider': 'ché me cague a trompadas de risa con el content, publicá más xfiiiiiiiii',
  'Elseroculto': '空间向量是线性代数研究的基石，这一学科在物理学、工程学、计算机科学和经济学等领域有广泛的应用。在这篇博客中，我们将深入探讨空间向量的世界，探索其基础、特性和应用。什么是空间向量？空间向量是一种数学结构，由一组称为向量的元素以及两种操作组成，即向量加法和标量乘法，满足某些基本性质。正式地，空间向量𝑉V 在域𝐹F 上是一个非空元素集合，称为向量，以及两种在𝑉V 上定义的操作：向量加法和标量乘法，这些操作满足八条公理，规定了它们必须满足的基本性质。特性和示例空间向量可以具有多种特性，其中一些包括对加法和标量乘法封闭、存在加法单位元素、存在加法逆元素和操作的结合律。常见的空间向量示例包括欧几里得空间𝑅𝑛，其中包含所有n-维向量，其坐标为实数，以及𝑃𝑛多项式空间，其多项式的次数不超过n。在现实生活中的应用空间向量在现实世界中有广泛的应用。在物理学中，它们用于建模力、力矩和电磁场。在工程学中，它们应用于结构分析、电路和控制系统。在计算机科学中，它们用于图像处理、计算机图形学和机器学习。甚至在经济学中，它们用于建模经济变量之间的关系和优化财务决策。高级特性和定理除了基本性质外，空间向量还表现出一系列更高级的特性和重要定理。例如，共同基础定理规定所有空间向量都有一个基础，即一组线性独立的向量，它们生成整个空间。秩定理告诉我们矩阵的秩等于可以从其列中获得的线性独立向量的最大数量。结论空间向量是数学和应用科学中强大而多才多艺的工具。它们的研究不仅提供了对空间结构和线性变换的深入理解，而且在各种领域都有实际应用。无论是对亚原子粒子运动建模还是设计人工智能算法，空间向量仍然是我们对周围世界理解的不可或缺的一部分。希望这次探索给您带来了对空间向量重要性和美妙之处的清晰认识！如果您有任何问题或评论，请随时分享！'},
 'https://josemamerto.blogspot.com/2024/05/pero-esto-que